In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import curve_fit
from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LinearRegression
from scipy.signal import bessel, sosfiltfilt

from sklearn.neighbors import KernelDensity
from data_manager import DataManager
import transport_signal_processing as tsp

In [2]:
def steps_detection(I, ws=5, n_iters=256):
    # envelope iterative smoothing
    Is = I.copy()
    for k in range(n_iters):
        Imax, Imin = tsp.signals.envelope(Is, ws)
        Is = 0.5*(Imax + Imin)

    # steps location
    ids_step = np.where(np.abs(np.diff(np.sign(np.diff(Is)))) > 0.5)[0]

    # steps amplitudes
    dI = np.abs(np.diff(Is[ids_step]))
    
    return Is, ids_step, dI

In [3]:
def steps_detection_spectrum(I, ws=5, n_order=10):
    # results variables
    Is_l, ids_step_l = [], []
    
    # envelope iterative smoothing
    Is = I.copy()
    for k in range(2**n_order):
        Imax, Imin = tsp.signals.envelope(Is, ws)
        Is = 0.5*(Imax + Imin)

        # if order reached
        if ((np.log2(k+1) - np.ceil(np.log2(k+1))) == 0):
            # steps location
            ids_step = np.where(np.abs(np.diff(np.sign(np.diff(Is)))) > 0.5)[0]
            
            # store results
            Is_l.append(Is.copy())
            ids_step_l.append(ids_step.copy())
    
    return Is_l, ids_step_l

In [4]:
# parameters
# voltages: 80, 100, 120, 150, 170, 190
path = "*"
re_sel = "AA00300AA"
key = "voltage"
level = 1
selected_only = True

# setup database connector
sigman = DataManager('database')

# load segments informations
sinfo_l = sigman.load_info(path, 's*')

# convert info to dataframe
df = pd.DataFrame(sinfo_l)

# keep only selected polymers in dataframe and signal info
df = df[df['polymer_name'].str.match(re_sel) & (df['selected'] > (level-1))]
sinfo_l = [sinfo_l[i] for i in df.index.values]
df = df.reset_index(drop=True)

# debug print
key_names = np.unique(df[key].values)
key_names = np.array(sorted(key_names.astype(int))).astype(np.str)  # numerical value only
print(key_names.shape[0], list(key_names))

# check
#assert len(polymer_names) == 1

KeyError: 'polymer_name'

In [ ]:
# batch compute average noramlized histograms
for vkey in key_names:
    # get segments information
    sinfo_l = df[df[key] == vkey].to_dict("records")
    
    # load events core
    cores = tsp.utils.load_core_events(sigman, sinfo_l, selected_only=True)
    
    # levels analysis for all events
    levels = []
    H_l = []
    for i in tqdm(range(len(cores))):
        # step detection
        core = cores[i]
        t = core[:,0]
        I = core[:,1]
        Is_l, ids_step_l = steps_detection_spectrum(I, ws=5, n_order=10)

        # plot
        H = []
        for k in range(len(Is_l)):
            h = np.histogram(Is_l[k], bins=40, range=(0.0,20.0))[0]
            H.append(h / np.sum(h))

        # aggregate distributions and find peaks
        H = np.stack(H)
        mh = np.mean(H, axis=0)
        x = np.linspace(0.0, 20.0, 40)
        ids_min, ids_max = tsp.signals.find_local_extrema(mh)

        # store results
        levels.append(x[ids_max])
        H_l.append(H)
        
    # pack and filter nan
    H_all = np.stack(H_l)
    H_all = H_all[~np.any(np.any(np.isnan(H_all), axis=2), axis=1)]
    H_all.shape

    # mean histogram
    mh = np.mean(np.mean(H_all, axis=0), axis=0)
    ids_min, ids_max = tsp.signals.find_local_extrema(mh)

    # debug print
    print("[{}], levels: {}".format(vkey, x[ids_max]))

    # plot
    plt.figure()
    plt.plot(x, mh, '.-')
    plt.plot(x[ids_min], mh[ids_min], 'd')
    plt.plot(x[ids_max], mh[ids_max], 's')
    plt.show()